<a href="https://colab.research.google.com/github/MarcoFierimonte/virtual-try-off/blob/main/Colab_Script_Fixes_for_Qwen_Clothing_Try_On_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This updated script implements the **Google Cloud Storage (GCS) Mounting** fix. This allows the 20GB+ model weights to be saved directly to your cloud bucket, bypassing the small local disk limit of the free Colab tier.

### Prerequisites:

1. **Create a GCS Bucket:** Go to your Google Cloud Console and create a bucket (e.g., `my-qwen-storage`).
2. **Hugging Face Token:** You will need a Read token from [huggingface.co/settings/tokens](https://huggingface.co/settings/tokens).

### New Colab Script:

In [ ]:
# ==========================================================
# 1. AUTHENTICATION & GCS MOUNTING
# ==========================================================
from google.colab import auth
import os

# LOGIN to Google to access your Cloud Storage
auth.authenticate_user()

# SET YOUR BUCKET NAME HERE
BUCKET_NAME = "your-unique-bucket-name" # <--- CHANGE THIS
MOUNT_PATH = "/content/qwen_data"

# Install gcsfuse to mount the bucket
!echo "deb http://packages.cloud.google.com/apt gcsfuse-focal main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!apt-get update
!apt-get install gcsfuse

# Create directory and mount the bucket
!mkdir -p {MOUNT_PATH}
!gcsfuse --implicit-dirs {BUCKET_NAME} {MOUNT_PATH}

# REDIRECT HUGGINGFACE CACHE TO THE BUCKET
# This ensures the 20GB model goes to the cloud, not the local disk
os.environ['HF_HOME'] = os.path.join(MOUNT_PATH, "hf_cache")

# ==========================================================
# 2. INSTALL DEPENDENCIES
# ==========================================================
!pip install -U git+https://github.com/huggingface/diffusers.git
!pip install -U transformers accelerate peft bitsandbytes sentencepiece gradio

import torch
from PIL import Image
from diffusers import QwenImageEditPlusPipeline
from diffusers.utils import load_image
import gradio as gr

# ==========================================================
# 3. SETUP MODEL (WITH 4-BIT OPTIMIZATION)
# ==========================================================
model_id = "Qwen/Qwen-Image-Edit-2509"

print("Loading model into GCS Bucket... This may take 10+ mins on the first run.")
# load_in_4bit=True is required to fit the 16GB VRAM of the T4 GPU
pipe = QwenImageEditPlusPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    load_in_4bit=True
)

# ==========================================================
# 4. DEFINE FUNCTIONS
# ==========================================================

def extract_clothes(input_image):
    # Load LoRA for extraction
    pipe.load_lora_weights(
        "JamesDigitalOcean/Qwen_Image_Edit_Extract_Clothing",
        weight_names="qwen_image_edit_remove_body.safetensors",
        adapter_name="removebody"
    )

    if isinstance(input_image, str):
        pil_image = load_image(input_image)
    else:
        pil_image = Image.fromarray(input_image).convert("RGB")

    prompt = "removebody remove the person from this image, but leave the outfit. the clothes should remain after deleting the person's body, skin, and hair. leave the clothes in front of a white background"

    image = pipe(
        image=[pil_image],
        prompt=prompt,
        num_inference_steps=30
    ).images[0]

    pipe.unload_lora_weights()
    return image

def tryon_clothes(clothing_image, person_image):
    # Load LoRA for try-on
    pipe.load_lora_weights(
        "JamesDigitalOcean/Qwen_Image_Edit_Try_On_Clothes",
        weight_names="qwen_image_edit_tryon.safetensors",
        adapter_name="tryonclothes"
    )

    pil_clothes = Image.fromarray(clothing_image).convert("RGB") if not isinstance(clothing_image, Image.Image) else clothing_image
    pil_person = Image.fromarray(person_image).convert("RGB") if not isinstance(person_image, Image.Image) else person_image

    prompt = "tryonclothes put the set of clothing onto the person in the image. Style the person in the image using the clothing provided."

    image = pipe(
        image=[pil_person, pil_clothes],
        prompt=prompt,
        num_inference_steps=30
    ).images[0]

    pipe.unload_lora_weights()
    return image

# ==========================================================
# 5. GRADIO UI
# ==========================================================
with gr.Blocks() as demo:
    gr.Markdown("# Qwen Image Edit: Clothing Try-On (GCS Powered)")
    gr.Markdown("Using Cloud Storage to bypass Colab Disk Limits.")

    with gr.Row():
        with gr.Column():
            input_model = gr.Image(label="1. Original Photo (Model wearing clothes)")
            extract_btn = gr.Button("Step 1: Extract Clothing")
        with gr.Column():
            extracted_output = gr.Image(label="Extracted Clothing (White Background)")

    with gr.Row():
        with gr.Column():
            target_person = gr.Image(label="2. Target Person (Photo to receive clothes)")
            tryon_btn = gr.Button("Step 2: Apply Try-On")
        with gr.Column():
            final_output = gr.Image(label="Final Result")

    extract_btn.click(fn=extract_clothes, inputs=input_model, outputs=extracted_output)
    tryon_btn.click(fn=tryon_clothes, inputs=[extracted_output, target_person], outputs=final_output)

demo.launch(debug=True, share=True)

### Why this works:

1. **`gcsfuse`**: This "tricks" the computer into thinking your Google Cloud bucket is just a folder.
2. **`os.environ['HF_HOME']`**: By pointing this variable to your bucket folder, Hugging Face will download all those massive model files (20GB+) directly into your cloud storage. Your Colab "Disk Space" bar will stay green.
3. **`load_in_4bit`**: This compresses the model so it fits into the 16GB VRAM of the free T4 GPU. Without this, you would get an "Out of Memory" (OOM) error immediately.